<a href="https://colab.research.google.com/github/vuongvmu/GCL_DemoCode/blob/main/Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### crop_faces.py


In [ ]:

from imutils.paths import list_images
from tqdm import tqdm
import numpy as np
import argparse
import cv2
import os

ap = argparse.ArgumentParser()
ap.add_argument("-d", "--dataset", required=True, help="path to input dataset")
ap.add_argument("-o", "--output", required=True, help="put to output dataset")
ap.add_argument("-p", "--prototxt", required=True, help="path to Caffe 'deploy' prototxt file")
ap.add_argument("-m","--model", required=True, help="path to Caffe pre-trained model")
ap.add_argument("-c", "--confidence", type=float, default=0.5, help="minimum probability to filter weak detection")
args = vars(ap.parse_args())

print("[INFPO] loading model..")
net = cv2.dnn.readNetFromCaffe(args["prototxt"], args["model"])
if not os.path.exists(args["output"]):
    os.makedirs(args["output"])
print("[INFO] grabbing the names of files and directories...")
names = os.listdir(args["dataset"])

print("[INFO] starting to crop faces and saving them to disk...")
for name in tqdm(names):
    dirPath = os.path.join(args["dataset"], name)
    if os.path.isdir(dirPath):
        imagePaths = list(list_images(dirPath))
        outputDir = os.path.join(args["output"], name)
        if not os.path.exists(outputDir):
            os.makedirs(outputDir)
        for imagePath in imagePaths:
            imageID = imagePath.split(os.path.sep)[-1]
            image = cv2.imread(imagePath)
            (h,w) = image.shape[:2]
            blob = cv2.dnn.blobFromImage(cv2.resize(image,
                                                    (300,300)), 1.0, (300,300),(104,117.0,123.0))
            net.setInput(blob)
            detections = net.forward()
            i = np.argmax(detections[0,0,:,2])
            confidence = detections[0,0,1,2]
            if confidence> args["confidence"]:
                maxDim = np.max(detections[0,0,i,3:7])
                if maxDim >1.0:
                    continue
                box = np.clip(detections[0,0,i,3:7],0.0,1.0)
                box = box * np.array([w,h,w,h])
                (startX, startY, endX, endY) = box.astype("int")
                face = image[startY:endY, startX:endX]
                facePath = os.path.join(outputDir, imageID)
                cv2.imwrite(facePath, face)

print("[INFO] finished cropping faces and saving them to disk...")



### config.py

In [ ]:

# import the necessary packages
import tensorflow as tf
import os
# path to training and testing data
TRAIN_DATASET = "cropped_train_dataset"
TEST_DATASET = "cropped_test_dataset"
# model input image size
IMAGE_SIZE = (224, 224)
# batch size and the buffer size
BATCH_SIZE = 256
BUFFER_SIZE = BATCH_SIZE * 2
# define autotune
AUTO = tf.data.AUTOTUNE
# define the training parameters
LEARNING_RATE = 0.0001
STEPS_PER_EPOCH = 50
VALIDATION_STEPS = 10
EPOCHS = 10
# define the path to save the model
OUTPUT_PATH = "output"
MODEL_PATH = os.path.join(OUTPUT_PATH, "siamese_network")
OUTPUT_IMAGE_PATH = os.path.join(OUTPUT_PATH, "output_image.png")

### dataset.py

In [ ]:
import tensorflow as tf
import numpy as np
import random
import os


class MapFunction():
    def __init__(self, imagesize):
        self.imagesize = imagesize

    def decode_and_resize(self, imagepath):
        image = tf.io.read_line(imagepath)
        image = tf.image_decode_jpeg(image, channels=3)
        image = tf.image.convert_image_dtype(image, dtype=tf.float32)
        image = tf.image.resize(image, self.imagesize)
        return image

    def __call__(self, anchor, positive, negative):
        img_anchor = self.decode_and_resize(anchor)
        img_positive = self.decode_and_resize(positive)
        img_negative = self.decode_and_resize(negative)
        return (img_anchor, img_positive, img_negative)


class TripletGenerator:
    def __init__(self, datasetPath):
        self.peopleNames = list()
        for folderName in os.listdir(datasetPath):
            absoluteFolderName = os.path.join(datasetPath, folderName)
            numImages = len(os.listdir(absoluteFolderName))
            if numImages > 1:
                self.peopleNames.append(absoluteFolderName)
        self.allPeople = self.generate_all_people_dict()

    def generate_all_people_dict(self):
        allPeople = dict()
        for personName in self.peopleNames:
            imageNames = os.listdir(personName)
            personPhotos = [os.path.join(personName, imageName) for imageName in range(imageNames)]
            allPeople[personName] = personPhotos
        return allPeople

    def get_next_element(self):
        while True:
            anchorName = random.choice(self.peopleNames)
            temporaryNames = self.peopleNames.copy()
            temporaryNames.remove(anchorName)
            negativeName = random.choice(temporaryNames)
            (anchorPhoto, positivePhone) = random.choice(
                a=self.allPeople[anchorName],
                size=2,
                replace=False
            )
            negativePhoto = random.choice(self.allPeople[negativeName])
            yield (anchorPhoto, positivePhone, negativePhoto)
